In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import sounddevice as sd
from scipy.io import wavfile
import os

# === Text-Audio Conversion Utilities ===
def text_to_bits(text):
    return np.array([int(b) for char in text for b in format(ord(char), '08b')])

def bits_to_text(bits):
    bits = bits[:len(bits) - len(bits) % 8]
    chars = [bits[i:i + 8] for i in range(0, len(bits), 8)]
    return ''.join([chr(int(''.join(str(b) for b in byte), 2)) for byte in chars])

def audio_to_bits(audio, sample_width=16):
    audio = np.clip(audio, -1.0, 1.0)
    scaled = (audio * 32767).astype(np.int16)
    return np.unpackbits(scaled.view(np.uint8))

def bits_to_audio(bits, sample_rate, sample_width=16):
    if len(bits) % 8 != 0:
        bits = bits[:len(bits) - len(bits) % 8]

    byte_data = np.packbits(bits)
    audio = np.frombuffer(byte_data, dtype=np.int16).astype(np.float32)
    audio /= 32768.0
    return audio, sample_rate


# === Hamming(7,4) Encoding and Decoding ===
def hamming_encode(data_bits):
    encoded = []
    for i in range(0, len(data_bits), 4):
        d = data_bits[i:i + 4]
        while len(d) < 4:
            d = np.append(d, 0)
        d1, d2, d3, d4 = d
        p1 = d1 ^ d2 ^ d4
        p2 = d1 ^ d3 ^ d4
        p3 = d2 ^ d3 ^ d4
        encoded += [p1, p2, d1, p3, d2, d3, d4]
    return np.array(encoded)

def hamming_decode(encoded_bits):
    decoded = []
    for i in range(0, len(encoded_bits), 7):
        block = encoded_bits[i:i + 7]
        if len(block) < 7:
            continue
        p1, p2, d1, p3, d2, d3, d4 = block
        s1 = p1 ^ d1 ^ d2 ^ d4
        s2 = p2 ^ d1 ^ d3 ^ d4
        s3 = p3 ^ d2 ^ d3 ^ d4
        error_pos = s1 + (s2 << 1) + (s3 << 2)
        if error_pos != 0 and error_pos <= 7:
            block[error_pos - 1] ^= 1
        decoded += [block[2], block[4], block[5], block[6]]
    return np.array(decoded)

# === Modulation/Demodulation ===
def bpsk_modulate(bits):
    return 2 * bits - 1  # BPSK maps 0 -> -1 and 1 -> 1

def bpsk_demodulate(signal):
    return (signal.real >= 0).astype(int)

def qpsk_modulate(bits):
    bits = np.pad(bits, (0, len(bits) % 2), constant_values=0)
    symbols = []
    for i in range(0, len(bits), 2):
        b1, b2 = bits[i], bits[i + 1]
        I = 1 if b1 == 1 else -1
        Q = 1 if b2 == 1 else -1
        symbols.append(I + 1j * Q)
    return np.array(symbols)

def qpsk_demodulate(symbols):
    bits = []
    for sym in symbols:
        bits.append(1 if sym.real >= 0 else 0)
        bits.append(1 if sym.imag >= 0 else 0)
    return np.array(bits)

def qam16_modulate(bits):
    bits = np.pad(bits, (0, 4 - len(bits) % 4), constant_values=0)
    mapping = {
        (0, 0): -3,
        (0, 1): -1,
        (1, 1): 1,
        (1, 0): 3
    }
    symbols = []
    for i in range(0, len(bits), 4):
        b1, b2, b3, b4 = bits[i:i + 4]
        I = mapping[(b1, b2)]
        Q = mapping[(b3, b4)]
        symbols.append(I + 1j * Q)
    return np.array(symbols)

def qam16_demodulate(symbols):
    bits = []
    for sym in symbols:
        I = sym.real
        Q = sym.imag
        bits += [1 if I > 0 else 0, 1 if abs(I) < 2 else 0]
        bits += [1 if Q > 0 else 0, 1 if abs(Q) < 2 else 0]
    return np.array(bits)

# === AWGN Channel ===
def add_awgn_noise(signal, EbN0_dB, bits_per_symbol):
    EbN0 = 10 ** (EbN0_dB / 10)
    signal_power = np.mean(np.abs(signal) ** 2)
    noise_power = signal_power / (2 * bits_per_symbol * EbN0)

    if np.iscomplexobj(signal):
        noise = np.sqrt(noise_power) * (np.random.randn(*signal.shape) + 1j * np.random.randn(*signal.shape))
    else:
        noise = np.sqrt(2 * noise_power) * np.random.randn(*signal.shape)

    return signal + noise



# === Plotting ===
def plot_signal(signal, title="Signal"):
    plt.figure(figsize=(10, 4))
    plt.plot(np.real(signal))
    plt.title(title)
    plt.xlabel("Sample")
    plt.ylabel("Amplitude")
    plt.grid(True)
    plt.tight_layout()
    plt.show()

# === Main Chat Loop ===
while True:
    print("\n--- Digital Communication Chat App ---")
    print("1. Text Message\n2. Audio File (.wav)\n3. Exit")
    mode = input("Enter choice (1, 2, or 3): ")

    if mode == '3':
        print("Exiting the chat app. Goodbye!")
        break

    # === Input Message or Audio ===
    if mode == '1':
        message = input("You: ")
        bits = text_to_bits(message)
        file_mode = 'text'
        sample_rate = None
    elif mode == '2':
        file_path = input("Enter path to .wav file: ").strip()
        if not os.path.exists(file_path):
            print("File not found.")
            continue
        sample_rate, audio = wavfile.read(file_path)
        if audio.ndim > 1:
            audio = audio[:, 0]  # Use only one channel
        audio = audio.astype(np.float32) / 32768
        bits = audio_to_bits(audio)
        file_mode = 'audio'
    else:
        print("Invalid option. Try again.")
        continue

    # === Hamming Encoding ===
    encoded = hamming_encode(bits)

    # === Modulation Type ===
    print("\nSelect Modulation:")
    print("1. BPSK\n2. QPSK\n3. 16-QAM")
    mod_choice = input("Enter choice (1/2/3): ")

    while True:
        try:
            EbN0_dB = float(input("Enter Eb/N0 in dB (recommended 2–10): "))
            if 0 <= EbN0_dB <= 20:
                break
            else:
                print("Value out of range.")
        except ValueError:
            print("Invalid input.")

    # === Modulate ===
    if mod_choice == '1':
        modulated = bpsk_modulate(encoded)
        bits_per_symbol = 1
    elif mod_choice == '2':
        modulated = qpsk_modulate(encoded)
        bits_per_symbol = 2
    elif mod_choice == '3':
        modulated = qam16_modulate(encoded)
        bits_per_symbol = 4
    else:
        print("Invalid modulation choice.")
        continue

    # === Transmit Over Channel ===
    noisy = add_awgn_noise(modulated, EbN0_dB, bits_per_symbol)

    # === Demodulate ===
    if mod_choice == '1':
        demodulated = bpsk_demodulate(noisy)
    elif mod_choice == '2':
        demodulated = qpsk_demodulate(noisy)
    elif mod_choice == '3':
        demodulated = qam16_demodulate(noisy)

    # === Decode ===
    decoded = hamming_decode(demodulated)

    # === Plot ===
    mod_name = ['BPSK', 'QPSK', '16-QAM'][int(mod_choice) - 1]
    plot_signal(modulated, f"Modulated Signal ({mod_name})")
    plot_signal(noisy, "Received Signal with AWGN")

    # === Output ===
    if file_mode == 'text':
        received_msg = bits_to_text(decoded)
        print(f"\n🛰️ Received Message: {received_msg}")
    elif file_mode == 'audio':
        received_audio, sr = bits_to_audio(decoded, sample_rate)
        if received_audio.size == 0:
            print("⚠️ Received audio is empty. Possibly due to noise or decoding errors.")
            continue

        received_audio = np.clip(received_audio, -1.0, 1.0)

        if np.max(np.abs(received_audio)) < 1e-3:
            print("⚠️ Received audio is nearly silent.")
        else:
            print("✅ Audio decoded successfully.")

        output_file = "received_output.wav"
        wavfile.write(output_file, sr, (received_audio * 32767).astype(np.int16))

        print(f"\n🔊 Playing received audio...")
        sd.play(received_audio, sr)
        sd.wait()
        print(f"✅ Saved to {output_file}")
